In [2]:
!pip install kagglehub[pandas-datasets]

In [3]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

df = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "yasserh/wine-quality-dataset",   # dataset handle (NO .csv)
    "WineQT.csv"                      # exact file name
)
df.drop(columns=['Id'],inplace=True)
df.head()


C:\Users\Yugesh\AppData\Local\Temp\ipykernel_20100\1896054038.py:4: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  df = kagglehub.load_dataset(


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


## L1 regularization with WineQt 

In [4]:
#importing libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.regularizers import l2,l1

In [5]:
x = df.drop(columns=['quality'])
y = df['quality']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42
)


In [7]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test) 


In [8]:
model = Sequential([
    Dense(64, activation='relu', kernel_regularizer=l1(0.01), input_shape=(11,)),
    Dense(32, activation='relu', kernel_regularizer=l1(0.01)),
    Dense(1)   # Regression output
])


c:\Users\Yugesh\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
model.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae']
)


In [10]:
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=16,
    verbose=0
)


In [11]:
loss, mae = model.evaluate(X_test, y_test)
print("Test MAE:", mae)

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.4479 - mae: 0.4802 
Test MAE: 0.474491149187088


In [18]:
new_data = df.drop('quality', axis=1).head(1).values
pred = model.predict(new_data)
pred

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step


array([[4.7294273]], dtype=float32)

In [24]:
pred_quality = np.round(pred).astype(int)
print(pred_quality)


[[5]]


In [19]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [23]:
df['quality'].unique()

array([5, 6, 7, 4, 8, 3], dtype=int64)

In [13]:
new_data

array([[ 7.4   ,  0.7   ,  0.    ,  1.9   ,  0.076 , 11.    , 34.    ,
         0.9978,  3.51  ,  0.56  ,  9.4   ,  5.    ]])

In [15]:
pred = model.predict(new_data)

ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense" is incompatible with the layer: expected axis -1 of input shape to have value 11, but received input with shape (1, 12)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(1, 12), dtype=float32)
  • training=False
  • mask=None